In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
    print("Own install")
    sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',
                     '../../../..',  '../../../../3rdparty',
                     '../../..',     '../../../3rdparty',
                     '../..',        '../../3rdparty',
                     '..',           '../3rdparty' ]
    
else: # In colab
    ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
    sys.path.append('./Jove')
    sys.path.append('./Jove/jove')

# -- common imports --
from jove.PcpJupyterNew import *
from jove.Def_TM     import *
from jove.Def_md2mc  import *
from jove.DotBashers import *
from jove.JoveEditor import *
from jove.AnimateTM import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

You may use any of these help commands:
help(step_tm)
help(run_tm)
help(explore_tm)

You may use any of these help commands:
help(md2mc)
.. and if you want to dig more, then ..
help(default_line_attr)
help(length_ok_input_items)
help(union_line_attr_list_fld)
help(extend_rsltdict)
help(form_delta)
help(get_machine_components)

You may use any of these help commands:
help(ResetStNum)
help(NxtStateStr)

 "help(JoveEditor)" gives you info on the generic animation panel that animates all machines. 


## Question 1 (25 points):

### Solve the PCP instance given below by hand first (15 points) 


### TILES = [('110', '1'), ('1', '0'), ('0', '110'),       ('0','1')] 

### And then by using the tool (10 points)

### using the command pcp_solve( TILES  , OWN_INSTALL)  

### You may always use help(pcp_solve) for details

# Hand Solution

### Express it as a sequence of tiles with repetition, numbering the first tile 1, second tile 2, etc.

### Example: If the solution is Tile-1 then Tile-3, then Tile-2, then Tile-2, write 1322

* Solution:  The series of tiles that I found to be a solution for this PCP is:
 * 14223
  * 110 | 0 | 1 | 1 |   0
  * 1   | 1 | 0 | 0 | 110

In [ ]:
# Tool Solution

In [3]:
pcp_solve([('110','1'), ('1', '0'), ('0', '110'), ('0','1')], OWN_INSTALL)

Detected platform linux
 Running the command ... :  ['./Jove/jove/pcp', '-i', 'temp.txt']
Solution(s) to PCP instance are below. Note: the tiles may be reversed,
  as the solver may sometimes present the solution in reverse.:

Solution 1
[3, 2, 2, 4, 1]
0   1   1   0   011 
011 0   0   1   1   




## Question 2 (25 points):

Part a (15 points): Adapt the proof of Theorem 15.3, Page 233, to a very similar proof that spells out that HaltTM is not recursive.  
HaltTM = {<M, w> : M is a TM and w its input and M halts on w}. Please imitate the proof of 15.3 step by step, making the necessary changes:

### Your proof below



Part b (10 points): Adapt the proof in Figure 15.10 to argue that CFL_TM is not recursive.  
CFL_TM = {\<M> : M is a TM whose language is context − free}

### Your proof below



## Question 3 (25 points):

- Design a deterministic Turing machine (DTM) over the input alphabet Σ = {a, b, #} for the language of all strings of the form w#x where w, x ∈ {a, b}∗ and w is a substring of x. See the definition of substring here: https://en.wikipedia.org/wiki/Substring

- You must design a deterministic Turing machine which absolutely has no non-determinism anywhere in its state transitions. A deterministic algorithm here requires searching for all w beginnings within x. A correct design will have a deterministic polynomial runtime.

- Add 5 more tests of your own.

In [40]:
Asg9DTM = md2mc('''
TM     

Iq0 : a ; X , R -> q1  !! Go right to start matching 'a's after the #
Iq0 : b ; Y , R -> q2
Iq0 : . ; . , R -> Iq0  !! If the 'w' string is nothing, epsilon will always be a substring of 'x', regardless of what it is
Iq0 : # ; # , R -> q9   !! The state that goes through the string until '.' which then transitions to a final state

q9  : 0 ; 0,R | 1 ; 1,R | 2 ; 2,R | a ; a,R | b ; b,R -> q9
q9  : . ; .,R -> F

q1 : a ; a , R -> q1
q1 : b ; b , R -> q1
q1 : # ; # , R -> q3  !! This state matches an 'a'

q2 : a ; a , R -> q2
q2 : b ; b , R -> q2
q2 : # ; # , R -> q4  !! This state matches a 'b'

q3 : b ; 0, L -> q7  !! Trying to match 'a', if 'b' sequence is broken, undo all progress and start over
q3 : 0 ; 0,R | 1 ; 1,R | 2 ; 2,R -> q3
q3 : a ; 1,L -> q5

q4 : a ; 0, L -> q7  !! Trying to match 'b', if 'a' sequence is broken, repeat like in the above
q4 : 0 ; 0, R | 1 ; 1,R | 2 ; 2,R -> q4    !! Traverse right until we arrive at an unused character
q4 : b ; 2 , L -> q5  !! Traverse back left until arrive at the number

q5 : a ; a , L -> q5 !!
q5 : b ; b , L -> q5 !! Don't know if I'll need these 2 states at the end
q5 : 2 ; 2 , L -> q5
q5 : 1 ; 1 , L -> q5
q5 : 0 ; 0 , L -> q5
q5 : # ; # , L -> q6  !! State that goes until the first number in the substring

q6 : a ; a , L | b ; b,L -> q6
q6 : X ; X,R | Y ; Y,R -> Iq0

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!! q7 and q8 are the reset states, because the section we were checking wasn't equivalent to
!!  the substring we have, we need to reset the state of the substring and start over
q7 : 0 ; 0,L | 1 ; 0,L | 2 ; 0,L -> q7
q7 : # ; #,L -> q8

q8 : X ; a,L | Y ; b,L | a ; a,L | b ; b,L -> q8
q8 : . ; .,R -> Iq0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


''')

### Test Cases:

In [31]:
help(explore_tm)

Help on function explore_tm in module jove.Def_TM:

explore_tm(T, tape, nsteps)
    A handy routine to print the result of run_tm plus making 
    future extensions to explore run-results.



In [42]:
explore_tm(Asg9DTM, "aba#aba",50)

Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('F', 8, 'XYX#121........', 18)
 via .. 
 ->('Iq0', 0, 'aba#aba', 50)
 ->('q1', 1, 'Xba#aba', 49)
 ->('q1', 2, 'Xba#aba', 48)
 ->('q1', 3, 'Xba#aba', 47)
 ->('q3', 4, 'Xba#aba', 46)
 ->('q5', 3, 'Xba#1ba', 45)
 ->('q6', 2, 'Xba#1ba', 44)
 ->('q6', 1, 'Xba#1ba', 43)
 ->('q6', 0, 'Xba#1ba', 42)
 ->('Iq0', 1, 'Xba#1ba', 41)
 ->('q2', 2, 'XYa#1ba', 40)
 ->('q2', 3, 'XYa#1ba', 39)
 ->('q4', 4, 'XYa#1ba', 38)
 ->('q4', 5, 'XYa#1ba', 37)
 ->('q5', 4, 'XYa#12a', 36)
 ->('q5', 3, 'XYa#12a', 35)
 ->('q6', 2, 'XYa#12a', 34)
 ->('q6', 1, 'XYa#12a', 33)
 ->('Iq0', 2, 'XYa#12a', 32)
 ->('q1', 3, 'XYX#12a', 31)
 ->('q3', 4, 'XYX#12a', 30)
 ->('q3', 5, 'XYX#12a', 29)
 ->('q3', 6, 'XYX#12a', 28)
 ->('q5', 5, 'XYX#121', 27)
 ->('q5', 4, 'XYX#121', 26)
 ->('q5', 3, 'XYX#121', 25)
 ->('q6', 2, 'XYX#121', 24)
 ->('Iq0', 3, 'XYX#121', 23)
 ->('q9', 4, 'XYX#121', 22)
 ->('q9', 5, 'XYX#121', 21)
 ->('q9', 6, 'XYX#121', 20)


In [41]:
explore_tm(Asg9DTM, "aba#ab",50)

Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('q3', 6, 'XYX#12', 28)
 via .. 
 ->('Iq0', 0, 'aba#ab', 50)
 ->('q1', 1, 'Xba#ab', 49)
 ->('q1', 2, 'Xba#ab', 48)
 ->('q1', 3, 'Xba#ab', 47)
 ->('q3', 4, 'Xba#ab', 46)
 ->('q5', 3, 'Xba#1b', 45)
 ->('q6', 2, 'Xba#1b', 44)
 ->('q6', 1, 'Xba#1b', 43)
 ->('q6', 0, 'Xba#1b', 42)
 ->('Iq0', 1, 'Xba#1b', 41)
 ->('q2', 2, 'XYa#1b', 40)
 ->('q2', 3, 'XYa#1b', 39)
 ->('q4', 4, 'XYa#1b', 38)
 ->('q4', 5, 'XYa#1b', 37)
 ->('q5', 4, 'XYa#12', 36)
 ->('q5', 3, 'XYa#12', 35)
 ->('q6', 2, 'XYa#12', 34)
 ->('q6', 1, 'XYa#12', 33)
 ->('Iq0', 2, 'XYa#12', 32)
 ->('q1', 3, 'XYX#12', 31)
 ->('q3', 4, 'XYX#12', 30)
 ->('q3', 5, 'XYX#12', 29)
 ->('q3', 6, 'XYX#12', 28)


In [43]:
explore_tm(Asg9DTM, "aba#baba",50)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('F', 17, '........XYX#0121........', 1)
 via .. 
 ->('Iq0', 0, 'aba#baba', 50)
 ->('q1', 1, 'Xba#baba', 49)
 ->('q1', 2, 'Xba#baba', 48)
 ->('q1', 3, 'Xba#baba', 47)
 ->('q3', 4, 'Xba#baba', 46)
 ->('q7', 3, 'Xba#0aba', 45)
 ->('q8', 2, 'Xba#0aba', 44)
 ->('q8', 1, 'Xba#0aba', 43)
 ->('q8', 0, 'Xba#0aba', 42)
 ->('q8', 7, '........aba#0aba', 41)
 ->('Iq0', 8, '........aba#0aba', 40)
 ->('q1', 9, '........Xba#0aba', 39)
 ->('q1', 10, '........Xba#0aba', 38)
 ->('q1', 11, '........Xba#0aba', 37)
 ->('q3', 12, '........Xba#0aba', 36)
 ->('q3', 13, '........Xba#0aba', 35)
 ->('q5', 12, '........Xba#01ba', 34)
 ->('q5', 11, '........Xba#01ba', 33)
 ->('q6', 10, '........Xba#01ba', 32)
 ->('q6', 9, '........Xba#01ba', 31)
 ->('q6', 8, '........Xba#01ba', 30)
 ->('Iq0', 9, '........Xba#01ba', 29)
 ->('q2', 10, '........XYa#01ba', 28)
 ->('q2', 11, '........XYa#01ba', 

In [44]:
explore_tm(Asg9DTM, "aba#babba",50)

Allocating  8  tape cells to the LEFT!
TM hasn't halted.
The truncated paths so far are as follows.
[('Iq0', 0, 'aba#babba', 50), ('q1', 1, 'Xba#babba', 49), ('q1', 2, 'Xba#babba', 48), ('q1', 3, 'Xba#babba', 47), ('q3', 4, 'Xba#babba', 46), ('q7', 3, 'Xba#0abba', 45), ('q8', 2, 'Xba#0abba', 44), ('q8', 1, 'Xba#0abba', 43), ('q8', 0, 'Xba#0abba', 42), ('q8', 7, '........aba#0abba', 41), ('Iq0', 8, '........aba#0abba', 40), ('q1', 9, '........Xba#0abba', 39), ('q1', 10, '........Xba#0abba', 38), ('q1', 11, '........Xba#0abba', 37), ('q3', 12, '........Xba#0abba', 36), ('q3', 13, '........Xba#0abba', 35), ('q5', 12, '........Xba#01bba', 34), ('q5', 11, '........Xba#01bba', 33), ('q6', 10, '........Xba#01bba', 32), ('q6', 9, '........Xba#01bba', 31), ('q6', 8, '........Xba#01bba', 30), ('Iq0', 9, '........Xba#01bba', 29), ('q2', 10, '........XYa#01bba', 28), ('q2', 11, '........XYa#01bba', 27), ('q4', 12, '........XYa#01bba', 26), ('q4', 13, '........XYa#01bba', 25), ('q4', 14, '........X

In [45]:
explore_tm(Asg9DTM, "bba#bbabba",50)

Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('F', 11, 'YYX#221bba........', 15)
 via .. 
 ->('Iq0', 0, 'bba#bbabba', 50)
 ->('q2', 1, 'Yba#bbabba', 49)
 ->('q2', 2, 'Yba#bbabba', 48)
 ->('q2', 3, 'Yba#bbabba', 47)
 ->('q4', 4, 'Yba#bbabba', 46)
 ->('q5', 3, 'Yba#2babba', 45)
 ->('q6', 2, 'Yba#2babba', 44)
 ->('q6', 1, 'Yba#2babba', 43)
 ->('q6', 0, 'Yba#2babba', 42)
 ->('Iq0', 1, 'Yba#2babba', 41)
 ->('q2', 2, 'YYa#2babba', 40)
 ->('q2', 3, 'YYa#2babba', 39)
 ->('q4', 4, 'YYa#2babba', 38)
 ->('q4', 5, 'YYa#2babba', 37)
 ->('q5', 4, 'YYa#22abba', 36)
 ->('q5', 3, 'YYa#22abba', 35)
 ->('q6', 2, 'YYa#22abba', 34)
 ->('q6', 1, 'YYa#22abba', 33)
 ->('Iq0', 2, 'YYa#22abba', 32)
 ->('q1', 3, 'YYX#22abba', 31)
 ->('q3', 4, 'YYX#22abba', 30)
 ->('q3', 5, 'YYX#22abba', 29)
 ->('q3', 6, 'YYX#22abba', 28)
 ->('q5', 5, 'YYX#221bba', 27)
 ->('q5', 4, 'YYX#221bba', 26)
 ->('q5', 3, 'YYX#221bba', 25)
 ->('q6', 2, 'YYX#221bba', 24)
 ->('Iq0', 3, 'YYX#221bba', 

### 5 Tests You Add:

In [46]:
# Test 1
explore_tm(Asg9DTM, "aba#babba",100)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('q4', 17, '........XYa#00001', 29)
 via .. 
 ->('Iq0', 0, 'aba#babba', 100)
 ->('q1', 1, 'Xba#babba', 99)
 ->('q1', 2, 'Xba#babba', 98)
 ->('q1', 3, 'Xba#babba', 97)
 ->('q3', 4, 'Xba#babba', 96)
 ->('q7', 3, 'Xba#0abba', 95)
 ->('q8', 2, 'Xba#0abba', 94)
 ->('q8', 1, 'Xba#0abba', 93)
 ->('q8', 0, 'Xba#0abba', 92)
 ->('q8', 7, '........aba#0abba', 91)
 ->('Iq0', 8, '........aba#0abba', 90)
 ->('q1', 9, '........Xba#0abba', 89)
 ->('q1', 10, '........Xba#0abba', 88)
 ->('q1', 11, '........Xba#0abba', 87)
 ->('q3', 12, '........Xba#0abba', 86)
 ->('q3', 13, '........Xba#0abba', 85)
 ->('q5', 12, '........Xba#01bba', 84)
 ->('q5', 11, '........Xba#01bba', 83)
 ->('q6', 10, '........Xba#01bba', 82)
 ->('q6', 9, '........Xba#01bba', 81)
 ->('q6', 8, '........Xba#01bba', 80)
 ->('Iq0', 9, '........Xba#01bba', 79)
 ->('q2', 10, '........XYa#01bba', 78)
 ->('q2', 11, '

In [47]:
# Test 2
explore_tm(Asg9DTM, "bbab#aabbababb",100)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('F', 23, '........YYXY#002212abb........', 3)
 via .. 
 ->('Iq0', 0, 'bbab#aabbababb', 100)
 ->('q2', 1, 'Ybab#aabbababb', 99)
 ->('q2', 2, 'Ybab#aabbababb', 98)
 ->('q2', 3, 'Ybab#aabbababb', 97)
 ->('q2', 4, 'Ybab#aabbababb', 96)
 ->('q4', 5, 'Ybab#aabbababb', 95)
 ->('q7', 4, 'Ybab#0abbababb', 94)
 ->('q8', 3, 'Ybab#0abbababb', 93)
 ->('q8', 2, 'Ybab#0abbababb', 92)
 ->('q8', 1, 'Ybab#0abbababb', 91)
 ->('q8', 0, 'Ybab#0abbababb', 90)
 ->('q8', 7, '........bbab#0abbababb', 89)
 ->('Iq0', 8, '........bbab#0abbababb', 88)
 ->('q2', 9, '........Ybab#0abbababb', 87)
 ->('q2', 10, '........Ybab#0abbababb', 86)
 ->('q2', 11, '........Ybab#0abbababb', 85)
 ->('q2', 12, '........Ybab#0abbababb', 84)
 ->('q4', 13, '........Ybab#0abbababb', 83)
 ->('q4', 14, '........Ybab#0abbababb', 82)
 ->('q7', 13, '........Ybab#00bbababb', 81)
 ->('q7', 12, '........Ybab#00bbababb

In [48]:
# Test 3
explore_tm(Asg9DTM, "b#",10)

Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('q4', 2, 'Y#', 8)
 via .. 
 ->('Iq0', 0, 'b#', 10)
 ->('q2', 1, 'Y#', 9)
 ->('q4', 2, 'Y#', 8)


In [49]:
# Test 4
explore_tm(Asg9DTM, "#babba",100)

Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('F', 7, '#babba........', 93)
 via .. 
 ->('Iq0', 0, '#babba', 100)
 ->('q9', 1, '#babba', 99)
 ->('q9', 2, '#babba', 98)
 ->('q9', 3, '#babba', 97)
 ->('q9', 4, '#babba', 96)
 ->('q9', 5, '#babba', 95)
 ->('q9', 6, '#babba', 94)
 ->('F', 7, '#babba........', 93)


In [56]:
# Test 5
explore_tm(Asg9DTM, "bb#abaabb",100)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('F', 18, '........YY#000022........', 20)
 via .. 
 ->('Iq0', 0, 'bb#abaabb', 100)
 ->('q2', 1, 'Yb#abaabb', 99)
 ->('q2', 2, 'Yb#abaabb', 98)
 ->('q4', 3, 'Yb#abaabb', 97)
 ->('q7', 2, 'Yb#0baabb', 96)
 ->('q8', 1, 'Yb#0baabb', 95)
 ->('q8', 0, 'Yb#0baabb', 94)
 ->('q8', 7, '........bb#0baabb', 93)
 ->('Iq0', 8, '........bb#0baabb', 92)
 ->('q2', 9, '........Yb#0baabb', 91)
 ->('q2', 10, '........Yb#0baabb', 90)
 ->('q4', 11, '........Yb#0baabb', 89)
 ->('q4', 12, '........Yb#0baabb', 88)
 ->('q5', 11, '........Yb#02aabb', 87)
 ->('q5', 10, '........Yb#02aabb', 86)
 ->('q6', 9, '........Yb#02aabb', 85)
 ->('q6', 8, '........Yb#02aabb', 84)
 ->('Iq0', 9, '........Yb#02aabb', 83)
 ->('q2', 10, '........YY#02aabb', 82)
 ->('q4', 11, '........YY#02aabb', 81)
 ->('q4', 12, '........YY#02aabb', 80)
 ->('q4', 13, '........YY#02aabb', 79)
 ->('q7', 12, '........YY#020

## Question 4 (25 points):


-  Design a non-deterministic Turing machine (NDTM) over the input alphabet Σ = {a, b, #} for the language of all strings of the form w#x where w, x ∈ {a, b}∗ and w is a substring of x. See the definition of substring here: https://en.wikipedia.org/wiki/Substring

- You must design a non-deterministic Turing machine—i.e., not design a DTM and call it an NDTM. You must employ a distinct phase of taking a guess (indicating it via your comments) and check the guess. A correct design will have a nondeterministic polynomial runtime, which means once the guess is made, only a polynomial zig-zag search must be involved to confirm the guess.

- Add 5 tests on your own.

In [ ]:
# w#x NDTM
Asg9NDTM = md2mc('''TM

Iq0 : . ; . , R -> F1  !! If the first thing is epsilon, the format is accepted
Iq0 : a ; a , S -> q1
Iq0 : b ; b , S -> q1

q1 : a ; 1 , L -> q2
q1 : a ; a , R -> q1
q1 : b ; b , R -> q1
q1 : b ; 2 , L -> q2


q2 : 0 ; 0,L | 1 ; 1,L -> q2
q2 : . ; ., R -> q3

q3 : a ; X , R -> 5
q3 : b ; Y , R ->

''')


### Test Cases:

In [ ]:
explore_tm(Asg9NDTM, "aba#aba",50)

In [ ]:
explore_tm(Asg9NDTM, "aba#ab",50)

In [ ]:
explore_tm(Asg9NDTM, "aba#baba",50)

In [ ]:
explore_tm(Asg9NDTM, "aba#babba",50)

In [ ]:
explore_tm(Asg9NDTM, "bba#bbabba",50)

### 5 Test Cases You Add

In [ ]:
# Test 1

In [ ]:
# Test 2

In [ ]:
# Test 3

In [ ]:
# Test 4

In [ ]:
# Test 5

# END
